In [6]:
# Maximum likelihood supervised classification

In [1]:
import numpy as np
from osgeo import gdal
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from shapely.geometry import mapping

In [3]:
# in_raster = r'E:\Desktop\Summary\种植结构提取-水稻\datas\CLIP\clip2021-08-22_4074_S2DL.tif'
# in_shp = r'E:\Desktop\Summary\种植结构提取-水稻\datas\MLSC\SHP\sample.shp'
# out_class = r'E:\Desktop\Summary\种植结构提取-水稻\datas\MLSC\MLSC0517_2021-08-22_4074_S2DL.tif'

In [13]:
in_raster = r'E:\Desktop\Summary\种植结构提取-水稻\whole_datas\2021-08-22_4074_S2DL.tif'
in_shp = r'E:\Desktop\Summary\种植结构提取-水稻\whole_datas\MLSC\SHP\sample.shp'
out_class = r'E:\Desktop\Summary\种植结构提取-水稻\whole_datas\MLSC\MLSC0517_2021-08-22_4074_S2DL.tif'

In [14]:
def get_train_data(in_raster, in_shp, attr='Id'):
    """获取训练数据集及其标签"""
    shp_df = gpd.GeoDataFrame.from_file(in_shp)
    with rio.open(in_raster) as src:
        band_count = src.count
        X = np.array([], dtype=np.int16).reshape(band_count, 0)
        y = np.array([], dtype=np.int16)
        for index, row in shp_df.iterrows():
            geo_feature = [mapping(row['geometry'])]
            clip_raster = mask(src, geo_feature, crop=True)[0]  # 提取roi
            clip_raster_nozero = clip_raster[:, ~np.all(clip_raster == 0, axis=0)]  # 提取非0像元
            #
            y = np.append(y, [row[attr]] * clip_raster_nozero.shape[1])
            X = np.hstack((X, clip_raster_nozero))
    return X, y


def read_tiff(path):
    """读取栅格数据"""
    # 参数类型检查
    if isinstance(path, gdal.Dataset):
        dataset = path
    else:
        dataset = gdal.Open(path)

    if dataset:
        im_width = dataset.RasterXSize  # 栅格矩阵的列数
        im_height = dataset.RasterYSize  # 栅格矩阵的行数
        im_bands = dataset.RasterCount  # 波段数
        im_proj = dataset.GetProjection()  # 获取投影信息
        im_geotrans = dataset.GetGeoTransform()  # 获取仿射矩阵信息
        im_data = dataset.ReadAsArray(0, 0, im_width, im_height)  # 获取数据
        return im_data, im_width, im_height, im_bands, im_geotrans, im_proj


def write_tiff(im_data, im_geotrans, im_proj, out_path=None, no_data_value=None, return_mode='TIFF'):
    """保存栅格数据"""
    # 保存类型选择
    if 'int8' in im_data.dtype.name or 'bool' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_Int16
    else:
        datatype = gdal.GDT_Float32
    # 矩阵波段识别
    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    elif len(im_data.shape) == 2:
        # 统一处理为三维矩阵
        im_data = np.array([im_data], dtype=im_data.dtype)
        im_bands, im_height, im_width = im_data.shape
    else:
        im_bands, (im_height, im_width) = 1, im_data.shape
    # 根据存储类型的不同，获取不同的驱动
    if out_path:
        dataset = gdal.GetDriverByName('GTiff').Create(out_path, im_width, im_height, im_bands, datatype)
    else:
        dataset = gdal.GetDriverByName('MEM').Create('', im_width, im_height, im_bands, datatype)
    # 写入数据
    if dataset is not None:
        dataset.SetGeoTransform(im_geotrans)
        dataset.SetProjection(im_proj)
    # 写入矩阵
    for i in range(im_bands):
        dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
        # 写入无效值
        if no_data_value is not None:
            # 当每个图层有一个无效值的时候
            if isinstance(no_data_value, list) or isinstance(no_data_value, tuple):
                if no_data_value[i] is not None:
                    dataset.GetRasterBand(i + 1).SetNoDataValue(no_data_value[i])
            else:
                dataset.GetRasterBand(i + 1).SetNoDataValue(no_data_value)
    # 根据返回类型的不同，返回不同的值
    if return_mode.upper() == 'MEMORY':
        return dataset
    elif return_mode.upper() == 'TIFF':
        del dataset
        return None

In [15]:
train_X, train_y = get_train_data(in_raster, in_shp)
print('x_shape:', train_X.shape)
print('y_shape:', train_y.shape)
print(train_y)
class_num = len(np.unique(train_y))  # 待分类类别个数
print(class_num)
class_label = np.unique(train_y)  # 待分类类别标签
print('标签:', class_label)
bands = train_X.shape[0]  # 波段数

x_shape: (23, 160997)
y_shape: (160997,)
[1 1 1 ... 0 0 0]
2
标签: [0 1]


In [ ]:
u, c = [], []
for i in class_label:
    label_index = np.argwhere(train_y == i)
    # print(label_index)
    label_index = label_index.flatten()
    # print(label_index)
    train_X_i = train_X[:, label_index]
    # print(train_X_i)
    u_i = np.mean(train_X_i, axis=1)
    # print(u_i)
    u_i = u_i.tolist()
    c_i = np.cov(train_X_i)
    u.append(u_i)
    c.append(c_i)
# 假设协方差阵相同
print('u', u)
print('u.shape', np.array(u).shape)
print('\n', '--------------------------')
print('c.shape', np.array(c).shape)
print('c', c)

In [17]:
c_all = 0
for i in range(class_num):
    c_all += c[i]
c_all = c_all[:19, :19]
# print(type(u))
np.linalg.inv(c_all)

a = np.array(u)
#
u = list(a[:,:19])
u

[array([2.17744485e+02, 2.64358952e+02, 6.27591341e+02, 3.88492462e+02,
        1.03341013e+03, 3.36760028e+03, 4.40961378e+03, 4.42301218e+03,
        4.67455926e+03, 4.64538595e+03, 2.35937933e+03, 1.19221209e+03,
        4.29690789e+02, 2.83451077e+03, 4.00171642e+00, 4.00186487e+01,
        6.42812957e+01, 2.73063693e+01, 4.63897200e-04]),
 array([4.46291918e+02, 5.22508660e+02, 8.12935323e+02, 6.16914820e+02,
        9.01331131e+02, 9.53302631e+02, 1.02825756e+03, 9.63536719e+02,
        9.77125851e+02, 9.85548649e+02, 7.04182236e+02, 4.72363697e+02,
        4.36608364e+02, 3.18052936e+03, 5.57927539e+00, 6.32406949e+01,
        8.31015367e+01, 5.35492370e+01, 6.64428722e-02])]

In [18]:
f = []
for i in range(class_num):
    C_i = np.dot(np.linalg.inv(c_all), np.array(u[i]))  # size:(3,1)
    C_oi = -0.5 * np.dot(np.array(u[i]).reshape(1, -1), C_i)  # size:(1,1)
    f.append([C_i, C_oi])
f

[[array([-8.25555890e-03, -6.86393510e-02, -2.00579704e-02,  7.43207774e-02,
         -1.45149425e-02, -4.24999902e-02,  2.56944251e-02,  1.22015855e-02,
         -6.63948110e-03, -1.62066665e-03,  8.03752015e-02, -6.11211953e-02,
          6.48490793e+00, -4.93315830e-03,  4.40728894e+01,  5.12468200e-01,
          1.37134846e-01, -3.82322642e-01, -1.71968805e+00]),
  array([-1525.73113633])],
 [array([ 2.27724081e-04, -6.36138522e-02, -2.56382623e-02,  7.07124444e-02,
         -1.08270042e-02, -3.47431203e-02,  2.60790375e-02,  1.20830067e-02,
         -1.22792604e-02, -2.78716920e-03,  7.40041730e-02, -5.62244049e-02,
          6.57449195e+00, -5.03094922e-03,  4.45592198e+01,  5.05578578e-01,
          9.99681933e-02, -3.22140539e-01, -1.76257755e+00]),
  array([-1561.0151535])]]

In [19]:
im_data, im_width, im_height, im_bands, im_geotrans, im_proj = read_tiff(in_raster)  # 读取待分类栅格数据
tmp = np.array(im_data)
tmp.shape

(23, 4841, 6368)

In [20]:
im_data=tmp[:19,:,:]
im_data.shape

(19, 4841, 6368)

In [21]:
rs_data = np.zeros((class_num, im_height, im_width))  # 新建像元属于每个类别下的概率数据
for i in range(class_num):
    tmp_data = im_data.transpose(1, 2, 0)
    rs_i = np.dot(tmp_data, f[i][0])
    rs_i = rs_i + f[i][1]
    rs_data[i, :, :] = rs_i
# 分类最大值index
rs_index = np.argmax(rs_data, axis=0)  # 提取最大概率所属的类别索引
# 分类index 对应的 分类标签label
label_dict = dict(zip(np.array([i for i in range(class_num)]), class_label))
rs_label = np.vectorize(label_dict.get)(rs_index)

In [22]:
write_tiff(rs_label, im_geotrans, im_proj, out_path=out_class)